In [1]:
import os
localfs_path = os.environ.get('SCRATCH_LOCAL') + '/'

In [2]:
import hail as hl

hl.init(
    tmp_dir=(localfs_path+'tmp_hail'),
    #spark_conf={'spark.driver.memory': '10G', 'spark.executor.memory': '5G'}, # I don't know what should be here
    default_reference='GRCh38'
) 

2023-04-17 11:54:52.749 WARN  NativeCodeLoader:60 - Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
Running on Apache Spark version 3.1.3
SparkUI available at http://ac0079:4040
Welcome to
     __  __     <>__
    / /_/ /__  __/ /
   / __  / _ `/ / /
  /_/ /_/\_,_/_/_/   version 0.2.109-b71b065e4bb6
LOGGING: writing to /net/pr2/projects/plgrid/plggneuromol/imdik-zekanowski-gts/preprocessing/oligogenic-model/hail-20230417-1154-0.2.109-b71b065e4bb6.log


In [3]:
from bokeh.io import show, output_notebook
from bokeh.layouts import gridplot
output_notebook()

Loading BokehJS ...

### select matrices for skat:
1. sportsmen vs GTS 40
2. polish zeros vs GTS 40
3. polish no zeros vs GTS 40
1. 1kg  vs GTS 40

In [6]:
mt_1 = hl.read_matrix_table(localfs_path+'polesunion.mt')

In [7]:
mt_2 = hl.read_matrix_table(localfs_path+'poles_zerosunion.mt')

In [8]:
mt_1.count() # this is for polish controls only

(3928354, 1788)

In [9]:
mt_2.count()

(10733526, 1788)

In [10]:
pheno = hl.import_table(
    '/net/pr2/projects/plgrid/plggneuromol/imdik-zekanowski-gts/data/pheno/GTS-coded-corrected-june-2021.csv',
    impute=True,
    delimiter=',',
    quote="\""
)

pheno = pheno.key_by(pheno.ID)

2023-04-12 16:15:52.051 Hail: INFO: wrote table with 187 rows in 1 partition to /localfs/2215677/tmp_hail/persist_tablergNdumcZGE
2023-04-12 16:15:53.368 Hail: INFO: Reading table to impute column types
2023-04-12 16:15:54.320 Hail: INFO: Finished type imputation
  Loading field 'ID' as type str (imputed)
  Loading field 'family' as type str (imputed)
  Loading field 'sex' as type str (imputed)
  Loading field 'kinship' as type str (imputed)
  Loading field 'disease' as type str (imputed)
  Loading field 'phenotype' as type str (imputed)
  Loading field 'add_pheno' as type str (imputed)
  Loading field 'heavy_tics' as type str (imputed)
  Loading field 'heavy_tics_familial' as type str (imputed)
  Loading field 'GTS_ASD_group' as type str (imputed)
  Loading field 'nonCTD' as type str (imputed)


In [11]:
mts = [mt_1, mt_2]

for idx, mt in enumerate(mts):
    mt = mt.annotate_cols(
    group = hl.if_else(
            mt.s.contains('B'),
            'local_controls',
            hl.if_else(
                (mt.s.contains('NA') | mt.s.contains('HG')),
                '1kg_controls',
                hl.if_else(
                    mt.s.contains('polish'),
                    'polish_controls',
                    'GTS'
                )
            )
        )
    )
    
    mt = mt.annotate_cols(phenotypes = pheno[mt.s])
    mts[idx] = mt

In [ ]:
#sportsmen vs GTS 40
s_vs_gts = mts[1].filter_cols((mts[1].group == 'local_controls') | (mts[1].phenotypes.family == '.'))
s_vs_gts = s_vs_gts.filter_rows(hl.agg.any(s_vs_gts.GT.is_non_ref()))
s_vs_gts = s_vs_gts.checkpoint(localfs_path+'s_vs_gts.mt')    

# polish no zeros vs GTS 40
p0_vs_gts = mts[0].filter_cols((mts[0].group == 'polish_controls') | (mts[0].phenotypes.family == '.'))
p0_vs_gts = p0_vs_gts.filter_rows(hl.agg.any(p0_vs_gts.GT.is_non_ref()))
p0_vs_gts = p0_vs_gts.checkpoint(localfs_path+'p0_vs_gts.mt') 

# polish zeros vs GTS 40
pw_vs_gts = mts[1].filter_cols((mts[1].group == 'polish_controls') | (mts[1].phenotypes.family == '.'))
pw_vs_gts = pw_vs_gts.filter_rows(hl.agg.any(pw_vs_gts.GT.is_non_ref()))
pw_vs_gts = pw_vs_gts.checkpoint(localfs_path+'pw_vs_gts.mt') 

# 1kg vs GTS 40
eur_vs_gts = mts[1].filter_cols((mts[1].group == '1kg_controls') | (mts[1].phenotypes.family == '.'))
eur_vs_gts = eur_vs_gts.filter_rows(hl.agg.any(eur_vs_gts.GT.is_non_ref()))
eur_vs_gts = eur_vs_gts.checkpoint(localfs_path+'eur_vs_gts.mt') 

In [10]:
def run_pca(mtx, mtx_path):
    for_pca = mtx.filter_rows(mtx.variant_qc.AF[1] > 0.05)
    pruned_variant_table = hl.ld_prune(for_pca.GT, r2=0.2, bp_window_size=500000)
    for_pca = for_pca.filter_rows(hl.is_defined(pruned_variant_table[for_pca.row_key]))

    for_pca = for_pca.checkpoint(localfs_path+'for_pca_20_'+mtx_path)
    for_pca = hl.read_matrix_table(localfs_path+'for_pca_'+mtx_path)
    eigenvalues, pcs, _ = hl.hwe_normalized_pca(for_pca.GT, k=20)
    
    mtx = mtx.annotate_cols(scores = pcs[mtx.s].scores)
    mtx = mtx.checkpoint(localfs_path+'after_pca_20_'+mtx_path)
    
    return(mtx)

In [ ]:
s_vs_gts = hl.read_matrix_table(localfs_path+'s_vs_gts.mt')    
p0_vs_gts = hl.read_matrix_table(localfs_path+'p0_vs_gts.mt') 
pw_vs_gts = hl.read_matrix_table(localfs_path+'pw_vs_gts.mt') 
eur_vs_gts = hl.read_matrix_table(localfs_path+'eur_vs_gts.mt') 

In [6]:
def run_pca_no_filter(mtx, mtx_path):
    
    for_pca = mtx.sample_rows(0.1)

    for_pca = for_pca.checkpoint(localfs_path+'subset_'+mtx_path)
    for_pca = hl.read_matrix_table(localfs_path+'subset_'+mtx_path)
    eigenvalues, pcs, _ = hl.hwe_normalized_pca(for_pca.GT, k=20)
    
    mtx = mtx.annotate_cols(scores = pcs[mtx.s].scores)
    mtx = mtx.checkpoint(localfs_path+'after_pca_no_filters_'+mtx_path)
    
    return(mtx)

In [7]:
mts = [s_vs_gts, p0_vs_gts, pw_vs_gts, eur_vs_gts]
mts_paths = ['s_vs_gts', 'p0_vs_gts', 'pw_vs_gts', 'eur_vs_gts']

In [ ]:
for idx, mt in enumerate(mts):
    mts[idx] = run_pca(mt, mts_paths[idx])

In [ ]:
for idx, mt in enumerate(mts):
    mts_2[idx] = run_pca_no_filter(mt, mts_paths[idx])

2023-04-14 11:44:32.662 Hail: INFO: wrote matrix table with 498271 rows and 142 columns in 1995 partitions to /localfs/2215677/subset_s_vs_gts
    Total size: 3.12 GiB
    * Rows/entries: 3.12 GiB
    * Columns: 1.13 KiB
    * Globals: 11.00 B
    * Smallest partition: 0 rows (20.00 B)
    * Largest partition:  1214 rows (8.80 MiB)
2023-04-14 11:45:37.092 Hail: INFO: hwe_normalize: found 493692 variants after filtering out monomorphic sites.
2023-04-14 11:45:41.390 Hail: INFO: pca: running PCA with 20 components... 1995]
2023-04-14 11:46:22.250 Hail: INFO: wrote table with 0 rows in 0 partitions to /localfs/2215677/tmp_hail_2/persist_tablefWYbsYCn9l
    Total size: 23.23 KiB
    * Rows: 0.00 B
    * Globals: 23.23 KiB
    * Smallest partition: N/A
    * Largest partition:  N/A


In [ ]:
# the european group for some reason did not compute, so I am recomputing it again
# this matrix has many more variant so I will subset it smaller

eur_vs_gts = hl.read_matrix_table(localfs_path+'eur_vs_gts.mt') 

for_pca = eur_vs_gts.sample_rows(0.05)

for_pca = for_pca.checkpoint(localfs_path+'subset_eur_vs_gts')
for_pca = hl.read_matrix_table(localfs_path+'subset_eur_vs_gts')
eigenvalues, pcs, _ = hl.hwe_normalized_pca(for_pca.GT, k=20)
    
eur_vs_gts = eur_vs_gts.annotate_cols(scores = pcs[eur_vs_gts.s].scores)
eur_vs_gts = eur_vs_gts.checkpoint(localfs_path+'after_pca_no_filters_eur_vs_gts')

2023-04-14 14:51:11.806 Hail: INFO: wrote matrix table with 424331 rows and 541 columns in 1995 partitions to /localfs/2215677/subset_eur_vs_gts
    Total size: 2.80 GiB
    * Rows/entries: 2.80 GiB
    * Columns: 3.38 KiB
    * Globals: 11.00 B
    * Smallest partition: 0 rows (20.00 B)
    * Largest partition:  1034 rows (7.37 MiB)
2023-04-14 14:52:41.929 Hail: INFO: hwe_normalize: found 422774 variants after filtering out monomorphic sites.
2023-04-14 14:52:45.832 Hail: INFO: pca: running PCA with 20 components... 1995]
2023-04-14 14:54:38.284 Hail: INFO: wrote table with 0 rows in 0 partitions to /localfs/2215677/tmp_hail_2/persist_tablejJclfNbAcV
    Total size: 87.92 KiB
    * Rows: 0.00 B
    * Globals: 87.92 KiB
    * Smallest partition: N/A
    * Largest partition:  N/A


In [ ]:
# at this point we have a two different PCA options - mts
# PCA with LD prunning and without LD prunning -mts_2

In [40]:
gtf = hl.import_table(
    '/net/pr2/projects/plgrid/plggneuromol/resources/genecode_43/gencode.v43.basic.annotation.gtf',
    delimiter = "\t",
    no_header = True,
    comment = "#"
)

# filter the gtf to only contain protein-coding genes with any GO term
gtf = gtf.filter((gtf.f2 == 'gene') & (gtf.f8.contains('protein_coding')))

gtf = gtf.select(
    gene_position = hl.locus_interval(
        gtf.f0,
        hl.int(gtf.f3),
        hl.int(gtf.f4),
        reference_genome='GRCh38'
    ),
    gene_symbol = gtf.f8.split("\"")[5]
)  

go_genes = hl.import_table(
    '/net/pr2/projects/plgrid/plggneuromol/resources/human-genes-with-GO-and-symbols'
)

go_genes = go_genes.key_by(go_genes['UniProtKB Gene Name symbol'])

gtf = gtf.key_by(gtf['gene_symbol'])
gtf = gtf.filter(
    hl.is_defined(
        go_genes[gtf.gene_symbol]
    )
)

# overall we are keeping 18303 genes. At this poin the within gene intervals are extended by 20kb each side
start = hl.if_else(
    gtf.gene_position.start.position <= 20000, 1, gtf.gene_position.start.position - 20000
)

contig_len = hl.contig_length(gtf.gene_position.start.contig, reference_genome='GRCh38') 

stop = hl.if_else(
    (contig_len - gtf.gene_position.end.position) <= 20000,
    contig_len,
    gtf.gene_position.end.position + 20000
)

gtf = gtf.annotate(
    interval_20kb =
    hl.locus_interval(
        gtf.gene_position.start.contig,
        start,
        stop,
        reference_genome='GRCh38'
    )
)

gtf = gtf.key_by(gtf.interval_20kb)

2023-04-15 18:36:02.837 Hail: INFO: wrote table with 286313 rows in 1 partition to /localfs/2215677/tmp_hail_3/persist_tableBE4iqPoUQt
2023-04-15 18:36:03.448 Hail: INFO: Reading table without type imputation
  Loading field 'f0' as type str (not specified)
  Loading field 'f1' as type str (not specified)
  Loading field 'f2' as type str (not specified)
  Loading field 'f3' as type str (not specified)
  Loading field 'f4' as type str (not specified)
  Loading field 'f5' as type str (not specified)
  Loading field 'f6' as type str (not specified)
  Loading field 'f7' as type str (not specified)
  Loading field 'f8' as type str (not specified)
2023-04-15 18:36:04.224 Hail: INFO: wrote table with 23805 rows in 1 partition to /localfs/2215677/tmp_hail_3/persist_tableDQUDpXtqZV
2023-04-15 18:36:04.700 Hail: INFO: Reading table without type imputation
  Loading field 'Gene stable ID' as type str (not specified)
  Loading field 'UniProtKB Gene Name symbol' as type str (not specified)


In [36]:
mts = [s_vs_gts, p0_vs_gts, pw_vs_gts, eur_vs_gts]
mts_2 = [s_vs_gts, p0_vs_gts, pw_vs_gts, eur_vs_gts]
mts_paths = ['s_vs_gts', 'p0_vs_gts', 'pw_vs_gts', 'eur_vs_gts']

In [37]:
for idx, mt in enumerate(mts):
    mts[idx] = hl.read_matrix_table(localfs_path+'after_pca_'+mts_paths[idx])
    mts_2[idx] = hl.read_matrix_table(localfs_path+'after_pca_no_filters_'+mts_paths[idx])

In [ ]:
# now annotate the standard PCA with filters with the other PCA (no filters)
# this is still lacking for the eur samples

for idx, mt in enumerate(mts[2:]): 
    mt = mt.annotate_rows(
        nearest_genes_20kb = hl.array(
            hl.set(
                gtf.index(mt.locus, all_matches=True)['gene_symbol']))
    )
    
    mt = mt.filter_rows(hl.agg.any(mt.GT.is_non_ref()))
    mt = mt.explode_rows(mt.nearest_genes_20kb)
    
    mt_2 = hl.read_matrix_table(localfs_path+'after_pca_no_filters_'+mts_paths[2:][idx])

    mt = mt.annotate_cols(scores_no_filter = mt_2.cols()[mt.s].scores)
    
    mt = mt.checkpoint(localfs_path+'for_full_skat_'+mts_paths[2:][idx])
    mts[2:][idx] = mt

2023-04-15 18:36:05.165 Hail: WARN: cols(): Resulting column table is sorted by 'col_key'.
    To preserve matrix table column order, first unkey columns with 'key_cols_by()'
2023-04-15 18:36:09.408 Hail: INFO: Ordering unsorted dataset with network shuffle
2023-04-15 18:36:09.929 Hail: INFO: Ordering unsorted dataset with network shuffle
2023-04-15 18:36:13.612 Hail: INFO: Ordering unsorted dataset with network shuffle


In [ ]:
#repartition the full skat's

s_vs_gts = hl.read_matrix_table(
    '/localfs/2215677/for_full_skat_s_vs_gts'
)

s_vs_gts.naive_coalesce(1000).write(
    '/net/pr2/projects/plgrid/plggneuromol/imdik-zekanowski-gts/data/mts/oligogenic-model/s_vs_gts.mt'
)


In [4]:
#repartition the full skat's

p0_vs_gts = hl.read_matrix_table(
    '/localfs/2215677/for_full_skat_p0_vs_gts'
)

p0_vs_gts.naive_coalesce(1000).write(
    '/net/pr2/projects/plgrid/plggneuromol/imdik-zekanowski-gts/data/mts/oligogenic-model/p0_vs_gts.mt'
)

In [4]:
# at this point I have two matrix-tables read:
#s_vs_gts = hl.read_matrix_table(
#    '/net/pr2/projects/plgrid/plggneuromol/imdik-zekanowski-gts/data/mts/oligogenic-model/s_vs_gts.mt'
#) this is written again later

p0_vs_gts = hl.read_matrix_table(
    '/net/pr2/projects/plgrid/plggneuromol/imdik-zekanowski-gts/data/mts/oligogenic-model/p0_vs_gts.mt'
)

##### annotate samples with sex

In [5]:
sport_females = ['B434','B437','B440','B447','B514','B519','B521']

In [6]:
s_vs_gts = s_vs_gts.annotate_cols(
    sex = hl.if_else(
        s_vs_gts.group == 'GTS',
        s_vs_gts.phenotypes.sex,
        hl.if_else(
            hl.literal(sport_females).contains(s_vs_gts.s),
            'F',
            'M'
        )
    )
)

In [7]:
p0_vs_gts = p0_vs_gts.annotate_cols(
    sex = hl.if_else(
        p0_vs_gts.group == 'GTS',
        p0_vs_gts.phenotypes.sex,
        p0_vs_gts.sex
    )
)

In [ ]:
#prapre the s_vs_gts to tests - make it smaller and save it outside localfs

In [ ]:
s_vs_gts = s_vs_gts.filter_rows(s_vs_gts.cadd.score_phred > 0)
s_vs_gts = s_vs_gts.checkpoint(localfs_path+'s_vs_gts_1.mt')

In [31]:
s_vs_gts = hl.read_matrix_table(localfs_path+'s_vs_gts_1.mt')

In [32]:
s_vs_gts.write(
    '/net/pr2/projects/plgrid/plggneuromol/imdik-zekanowski-gts/data/mts/oligogenic-model/s_vs_gts.mt'
)

2023-04-16 21:56:57.586 Hail: INFO: wrote matrix table with 5520376 rows and 142 columns in 998 partitions to /net/pr2/projects/plgrid/plggneuromol/imdik-zekanowski-gts/data/mts/oligogenic-model/s_vs_gts.mt


## (Jacek) Run SKAT for sportsmen and GTS on scores, for all genes:

In [4]:
def run_skat_log(mtx, pcs, field):
    
    scores = [mtx[field][x] for x in list(range(pcs))]
    is_male = hl.int(mtx.sex == 'M')
    
    skat_table = hl.skat(key_expr=mtx.nearest_genes_20kb,
                         weight_expr=mtx.cadd.score_phred,
                         y=(mtx.group == 'GTS'),
                         x=mtx.GT.n_alt_alleles(),
                         covariates=[1, is_male] + scores,
                         max_size = 2500,
                         logistic = True)

    skat_table.order_by(skat_table.p_value).show(20)
    
    qq_plot = hl.plot.qq(
        skat_table.p_value
    )
    show(qq_plot)
    
    return(skat_table, qq_plot)

In [5]:
s_vs_gts = hl.read_matrix_table(
    '/net/pr2/projects/plgrid/plggneuromol/imdik-zekanowski-gts/data/mts/oligogenic-model/s_vs_gts.mt'
)

In [ ]:
skat_tables = []
plots = []

for pc in list(range(20)):
    
    print('tryin to run with '+str(pc)+' PCs and the pruned scores option')
    
    try:
        skat_table, qq_plot = run_skat_log(s_vs_gts, pc, 'scores')
        skat_tables.append(skat_table)
        plots.append(qq_plot)
        
    except Exception as e:
        print('run with'+str(pc)+' PCs and the pruned scores option failed')
        print(type(e))
    
    print('trying run with '+str(pc)+' PCs and the just subsetet scores')
    
    try:
        skat_table_no_filt, qq_plot_no_filt = run_skat_log(s_vs_gts, pc, 'scores_no_filter')
        skat_tables.append(skat_table_no_filt)
        plots.append(qq_plot_no_filt)
        
    except Exception as e:
        print('run with'+str(pc)+' PCs and subseted scores no filters failed')
        print(type(e))

        

tryin to run with 0 PCs and the pruned scores option


2023-04-17 11:59:20.011 Hail: INFO: Ordering unsorted dataset with network shuffle
2023-04-17 12:00:38.377 Hail: INFO: wrote table with 17847 rows in 998 partitions to /localfs/2325106/tmp_hail/persist_tableGzurS9hLg4
    Total size: 574.43 KiB
    * Rows: 574.42 KiB
    * Globals: 11.00 B
    * Smallest partition: 17 rows (477.00 B)
    * Largest partition:  23 rows (760.00 B)
2023-04-17 12:00:39.316 Hail: INFO: Ordering unsorted dataset with network shuffle


,,,,
id,size,q_stat,p_value,fault
str,int32,float64,float64,int32
"""GOLGA8Q""",10,7.32e+02,-2.27e-07,0
"""KCNK4""",148,1.47e+04,-9.74e-08,0
"""TPO""",208,8.80e+03,-8.90e-08,0
"""ZNF429""",393,2.52e+04,-8.62e-08,0
"""RRN3""",4,3.18e+02,-5.26e-08,0
"""UBE2A""",215,1.31e+04,-5.21e-08,0
"""DEFB106A""",58,1.94e+03,-4.21e-08,0
"""ANKRD36""",741,1.80e+04,-3.56e-08,0


2023-04-17 12:00:42.050 Hail: INFO: Ordering unsorted dataset with network shuffle
2023-04-17 12:01:05.981 Hail: INFO: wrote table with 17847 rows in 988 partitions to /localfs/2325106/tmp_hail/persist_tableGWwDzMDurG


## Now run SKAT on selected genes only

In [ ]:
gene_list = ['DCC', 'RBFOX', 'SLC30A9', 'DCAF4L1', 'SORCS3', 'KCNQ5', 'KCNQ-IT1', 'APOPT1', 'C14orf2', 'NAA11', 'NEGR1',
        'CHADL', 'SOX5', 'PANK2', 'COL27A1', 'PDGFB', 'CELSR3', 'OPA1', 'FBN2', 'WWC1', 'NIPBL', 
             'FN1', 'FBN2', 'SLITRK1', 'SLITRK2', 'SLITRK3', 'SLITRK4', 'SLITRK5', 'SLITRK6', 
             'HDC', 'OPRK1', 'PCDH10', 'NTSR2', 'CHD8', 'SCUBE1', 'PNKD', 'CNTNAP2', 'MOG', 
             'DRD2', 'DRD3', 'DRD4', 'DRD5', 'DAT1', 'DBH', 'HTR2A', 'TPH2', 'EAAT1', 'SAPAP3',
            'CTNNA3', 'NLGN4', 'FSCB', 'IMMP2L', 'NRXN1', 'AADAC', 'DBH', 'MAOA', 'HTR1A', 'HTR2C', 'SLC6A4',
             'TPH2', 'COL27A1', '5-HTTLPR', 'EAAT1', 'COL8A1', 'KCNE1', 'KCNE2',
         'RICTOR', 'WWC1', 'CELSR3, NIPBL', 'FN1', 'PNKD', 'CDH26', 'CADM2', 'OPCML', 'CDH9',
         'NCAM2', 'CD47', 'CDH5', 'CADM4', 'C1QBP', 'CTTN', 'LSAMP',
         'PKP4', 'PCDH1', 'CNTNAP2', 'MBP', 'GABBR2', 'GABBR2', 'GRIK4', 'NCR1', 'FLT3', 'IL12A', 'HDAC9',
         'CD180', 'CDH26', 'NCAM2', 'NTM', 'ROBO2', 'NR2F1', 'ASH1L', 'FLT3']

In [ ]:
#for idx, mt in enumerate(mts): 
#    mt = mt.annotate_rows(
#        nearest_genes_20kb = hl.array(
#            hl.set(
#                gtf.index(mt.locus, all_matches=True)['gene_symbol']))
#    )
    
#    mt = mt.filter_rows(hl.any(lambda x: hl.literal(gene_list).contains(x), mt.nearest_genes_20kb))
#    mt = mt.filter_rows(hl.agg.any(mt.GT.is_non_ref()))
#    mt = mt.explode_rows(mt.nearest_genes_20kb)
#    mt = mt.filter_rows(hl.literal(gene_list).contains(mt.nearest_genes_20kb))
#    mt = mt.checkpoint(localfs_path+'for_skat_'+mts_paths[idx])
#    mts[idx] = mt

In [ ]:
#at this point run some QC to compare the groups (overall) - do this later